In [2]:
from psaw import PushshiftAPI
import pandas as pd
import numpy as np
import datetime as dt
from sys import platform
api = PushshiftAPI()
pd.options.mode.chained_assignment = None  # default='warn'

In [3]:
#Easy switching between Windows and Mac directories
if platform == 'darwin':
    ticker = pd.read_csv(r'/Users/nishaldave/OneDrive - University of Bristol/Dissertation/Volume Forecasting/Data/WSB Scraper/tickers.csv',header=None)
    ticker[1] = '$'+ticker[0]
    ticker.to_csv(r'/Users/nishaldave/OneDrive - University of Bristol/Dissertation/Volume Forecasting/Data/WSB Scraper/tickers_new.csv', index=False)
    import csv
    with open(r'/Users/nishaldave/OneDrive - University of Bristol/Dissertation/Volume Forecasting/Data/WSB Scraper/tickers_new.csv') as csv_file:
        reader = csv.reader(csv_file, delimiter=',')
        tickers = list(reader)
    del tickers[0]
    
elif platform =='win32':
    ticker = pd.read_csv(r'C:\Users\Nish\OneDrive - University of Bristol\Dissertation\Volume Forecasting\Data\WSB Scraper\tickers.csv',header=None)
    ticker[1] = '$'+ticker[0]
    ticker.to_csv(r'C:\Users\Nish\OneDrive - University of Bristol\Dissertation\Volume Forecasting\Data\WSB Scraper\tickers_new.csv', index=False)
    import csv
    with open(r'C:\Users\Nish\OneDrive - University of Bristol\Dissertation\Volume Forecasting\Data\WSB Scraper\tickers_new.csv') as csv_file:
        reader = csv.reader(csv_file, delimiter=',')
        tickers = list(reader)
    del tickers[0]

In [ ]:
dfs = []

In [ ]:
#Scrapes all reddit submissions associated with each ticker 
for i in tickers:
    start_date = int(dt.datetime(2020, 6, 29, 0,0,0).timestamp())
    start_date2 = int(dt.datetime(2020, 6, 29, 23,59,59).timestamp())
    end_date = int(dt.datetime(2021, 6, 25, 0,0,0).timestamp())

    gen_init = api.search_submissions(subreddit='wallstreetbets',q=i,limit=10000,after=start_date,before=start_date2)
    df_init = pd.DataFrame([X.d_ for X in gen_init])
    df_final = df_init[0:0]

    while start_date < end_date:    
      gen_data = api.search_submissions(subreddit='wallstreetbets',q=i,limit=10000,after=start_date,before=start_date2)
      df_inter = pd.DataFrame([X.d_ for X in gen_data])
      df_final = pd.concat([df_final,df_inter],axis=0)
      #df_final['created_utc'] = pd.to_datetime(df_final['created_utc'],unit='s')
      df_final['asset'] = i[0]
      start_date  = start_date  + 86400
      start_date2 = start_date2 + 86400
      
    dfs.append(df_final)

In [ ]:
#Necessary variables to retain
#created_utc, link_flair_css_class, num_comments, selftext, title, asset.
for i in range(len(dfs)):
    dfs[i] = dfs[i][['created_utc','link_flair_css_class','num_comments','selftext','title','asset']]
    dfs[i]['created_utc'] = pd.to_datetime(dfs[i]['created_utc'],unit='s')
    dfs[i] = dfs[i].sort_values(by='created_utc',ascending=True)
    dfs[i] = dfs[i][dfs[i]['selftext'] != '[removed]']
    dfs[i] = dfs[i][dfs[i]['selftext'] != '[deleted]']
    dfs[i] = dfs[i][dfs[i]['selftext'].notna()]
    dfs[i] = dfs[i][dfs[i]['selftext']!=''] #drop selftexts where there is no valid description.

In [ ]:
pd.concat(dfs).to_csv('C:\Users\Nish\OneDrive - University of Bristol\Dissertation\Volume Forecasting\Data\WSB Scraper\Submissions\assets_v1s.csv')